In [2]:
import ccxt
import numpy as np
import pandas as pd
import time, datetime
import threading
import configparser
from glob import glob
import os
from math import ceil

from data_provider import get_symbols, read_data_path, market_to_symbol, create_klines

In [3]:
import gym

In [4]:
env = gym.make('CartPole-v0')

[2019-01-01 12:01:26,325] Making new env: CartPole-v0


In [5]:
env.reset()

array([ 0.03801261,  0.02955956, -0.04497569, -0.01916716])

In [6]:
env.render()

In [1]:
import tensorflow as tf

ImportError: cannot import name 'abs'

In [6]:
exchange = ccxt.bitfinex({'rateLimit': 20000})
exchange.timeframes.values()
exchange.enableRateLimit = True
exchange.rateLimit = 30000
exchange.load_markets()

{'AGI/BTC': {'active': True,
  'base': 'AGI',
  'baseId': 'AGI',
  'fee_loaded': False,
  'id': 'AGIBTC',
  'info': {'expiration': 'NA',
   'initial_margin': '30.0',
   'margin': False,
   'maximum_order_size': '200000.0',
   'minimum_margin': '15.0',
   'minimum_order_size': '100.0',
   'pair': 'agibtc',
   'price_precision': 5},
  'limits': {'amount': {'max': 200000.0, 'min': 100.0},
   'cost': {'max': None, 'min': 0.001},
   'price': {'max': 100000.0, 'min': 1e-05}},
  'lot': 1e-05,
  'maker': 0.001,
  'percentage': True,
  'precision': {'amount': 5, 'price': 5},
  'quote': 'BTC',
  'quoteId': 'BTC',
  'symbol': 'AGI/BTC',
  'taker': 0.002,
  'tierBased': True,
  'tiers': {'maker': [[0, 0.001],
    [500000, 0.0008],
    [1000000, 0.0006],
    [2500000, 0.0004],
    [5000000, 0.0002],
    [7500000, 0],
    [10000000, 0],
    [15000000, 0],
    [20000000, 0],
    [25000000, 0],
    [30000000, 0]],
   'taker': [[0, 0.002],
    [500000, 0.002],
    [1000000, 0.002],
    [2500000, 0.002]

In [5]:
exchange = ccxt.binance()
pair = 'BTC'
from_datetime = '2018-03-01 00:00:00'
#to_datetime = '2018-04-11 12:00:00'
#to_timestamp = exchange.parse8601(to_datetime)
now = exchange.milliseconds()
to_timestamp = now

timeframes = {'1m':1, '3m':3, '5m':5, '15m':15, '30m':30, '1h':60}
#exchange.timeframes.values()

symbol_list = get_symbols(exchange, pair)
markets = ['EOS-BTC', 'ONT-BTC', 'TRX-BTC']

data_path = read_data_path() + '/' + pair

In [6]:
#Обновление данных
if not os.path.isdir(data_path):
    print("Директория отсутствует")

start_timestamp = exchange.parse8601('2018-03-01 00:00:00')
print(start_timestamp)
start_time = time.time()

for market in markets:
    print(market)
    symbol = market_to_symbol(market)
    folders_exist = next(os.walk(data_path))[1]

    if market not in folders_exist:
        newpath = data_path + '/' + market
        os.makedirs(newpath)

    folders_exist = next(os.walk(data_path + '/' + market))[1]

    tf_keys = list(timeframes.keys())
    for tf in tf_keys[:6]:
        print(tf)
        tf_path = data_path + '/' + market +'/' + tf 
        if tf not in folders_exist:
            os.makedirs(tf_path)
        
        files = glob(tf_path + '/*.csv')
        from_timestamp = start_timestamp
        if files != []:
            df_old   = pd.read_csv(files[-1])
            from_timestamp = int(df_old.iloc[-1]['T'])
        else:
            df_old   = pd.DataFrame(columns = ['T', 'O', 'H', 'L', 'C', 'V'])
            
        data = []        
        while from_timestamp < to_timestamp:
            time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
            ohlcvs = exchange.fetch_ohlcv (symbol, tf, from_timestamp)    
            first  = ohlcvs[0][0]
            last   = ohlcvs[-1][0]
            #print('\t First candle epoch', first, exchange.iso8601(first))
            #print('\t Last candle epoch', last, exchange.iso8601(last), '\n')

            from_timestamp = first  # На случай если на заданную дату данных еще нет
            from_timestamp += len(ohlcvs) * timeframes[tf] * 1000 * 60
            data += ohlcvs

        if data != []:
            df_new = pd.DataFrame(data)
            df_new.columns = ['T', 'O', 'H', 'L', 'C', 'V']
            df = pd.concat([df_old, df_new])
            df = df.drop_duplicates()
            file_name = tf_path + '/' + market + '_' + str(last) + '.csv'
            df.to_csv(file_name, index = False)  
        
        files = glob(tf_path + '/*.csv')
        if len(files) > 1 : 
            os.remove(files[0])
    
print('Done')
time_taken = (time.time() - start_time)/60/60
print("Time taken = {0:.3f}".format(time_taken),' hours')

1519862400000
EOS-BTC
1m
3m
5m
15m
30m
1h
ONT-BTC
1m
3m
5m
15m
30m
1h
TRX-BTC
1m
3m
5m
15m
30m
1h
Done
Time taken = 0.288  hours


In [3]:
tf = 24
create_klines(data_path, markets, tf)

In [6]:
if not os.path.isdir(data_path):
    print("Директория отсутствует")


start_time = time.time()

for market in markets:
    print(market)
    symbol = market_to_symbol(market)
    folders_exist = next(os.walk(data_path))[1]

    if market not in folders_exist:
        newpath = data_path + '/' + market
        os.makedirs(newpath)

    folders_exist = next(os.walk(data_path + '/' + market))[1]

    tf_keys = list(timeframes.keys())
    for tf in tf_keys[:6]:
        print(tf)
        if tf not in folders_exist:
            newpath = data_path + '/' + market +'/' + tf 
            os.makedirs(newpath)

        data = []
        from_timestamp = exchange.parse8601(from_datetime)
        while from_timestamp < to_timestamp:
            time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
            ohlcvs = exchange.fetch_ohlcv (symbol, tf, from_timestamp)    
            first = ohlcvs[0][0]
            last = ohlcvs[-1][0]
            #print('\t First candle epoch', first, exchange.iso8601(first))
            #print('\t Last candle epoch', last, exchange.iso8601(last), '\n')

            from_timestamp = first  # На случай если на заданную дату данных еще нет
            from_timestamp += len(ohlcvs) * timeframes[tf] * 1000 * 60
            data += ohlcvs


        df = pd.DataFrame(data)
        df.columns = ['T', 'O', 'H', 'L', 'C', 'V']
        file_name = data_path + '/' + market + '/' +  tf + '/' + market + '_' + str(last) + '.csv'
        df.to_csv(file_name, index = False)    
    
print('Done')
time_taken = (time.time() - start_time)/60
print("Time taken = {0:.3f}".format(time_taken),' min')

EOS-BTC
1m
3m
5m
15m
30m
1h
TRX-BTC
1m
3m
5m
15m
30m
1h
Done
Time taken = 0.166  hours


In [ ]:
def cash_data_read(exchange, hold = 30):
    data_path = read_data_path()
    cash_data_path = data_path + '/Cash'
    
       
    run = True
    while run:
        now = exchange.milliseconds()
        df = pd.DataFrame({'timestamp':[now], 'time':[exchange.iso8601(now)]})
        file_name = cash_data_path + '/timestamp.csv'
        df.to_csv(file_name, index = False)
        
        df = pd.read_csv("markets_in_trade.csv")
        for row in df.iterrows():
            time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds

            market = row[1]['market']
            tf = row[1]['tf']
            symbol = market_to_symbol(market)
            try:
                ohlcvs =  exchange.fetch_ohlcv (symbol, tf)
            except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:
                error = exchange.iso8601(now), 'Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...'
                error = ' '.join(error)
                with open(cash_data_path +'/' + test.txt, 'a') as myfile:
                    myfile.write(error)
                time.sleep(hold)  
            
            
            df = pd.DataFrame(ohlcvs)
            df.columns = ['T', 'O', 'H', 'L', 'C', 'V']
            file_name = cash_data_path + '/' + market + '_' + tf + '.csv'
            df.to_csv(file_name, index = False) 
            
      

In [ ]:
a= ['a', 'b', 'c']
' '.join(a)

In [ ]:
   
def data_read(exchange, tick, restart = 0):
    data_path = read_data_path()

    with open("currencies.txt", 'r') as file:
        content = file.read()
        saving_currencies = content.split("\n") #split it into lines
    file.closed

    #Проверяем наличие директорий для сохранения файлов, создаем в случае отсутсвия
    if not os.path.isdir(data_path):
        print("Директория отсутствует")
        return 0
        
    folders_exist = next(os.walk(data_path))[1]
    folders_to_make = list(set(saving_currencies) - set(folders_exist))
    
    for folder in folders_to_make :
        newpath = data_path + folder +'/' + tick 
        os.makedirs(newpath)
        
    
    #Читаем и записываем данные
    dp = Bittrex(None, None, api_version= API_V2_0)

    for market_name in saving_currencies:
        print(market_name)
        folders_exist = next(os.walk(data_path + '/' + market_name))[1]
        if not (tick in folders_exist):
            newpath = data_path + market_name +'/' + tick 
            os.makedirs(newpath)
            
        dj = dp.get_candles(market_name, tick)
        df = json_normalize(dj['result'])
        last_time = df[-1:]['T'].values[0]
        last_time = last_time.replace(':','-')
        file_name = data_path + market_name + '/' +  tick + '/' + market_name + "-" + last_time + ".csv"
        df.to_csv(file_name, index = False)